In [7]:
import numpy as np
import cv2
from keras.preprocessing import image
from keras.models import model_from_json

In [2]:
face_cascade = cv2.CascadeClassifier('C:\\Users\\Vyshnave\\Anaconda3\\Lib\\site-packages\\opencv\\sources\\data\\haarcascades\\haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('C:\\Users\\Vyshnave\\Anaconda3\\Lib\\site-packages\\opencv\\sources\\data\\haarcascades\\haarcascade_eye.xml')
#smile_cascade = cv2.CascadeClassifier('C:\\Users\\Vyshnave\\Anaconda3\\Lib\\site-packages\\opencv\\sources\\data\\haarcascades\\haarcascade_smile.xml')

In [3]:
cap = cv2.VideoCapture(0)

In [4]:
model = model_from_json(open("facial_expression_model_structure.json", "r").read())
model.load_weights('facial_expression_model_weights.h5')

In [5]:
emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

In [6]:
while (True): 
 
    ret, img = cap.read() 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
 
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2) 
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
 
        eyes = eye_cascade.detectMultiScale(roi_gray) 
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,127,255),2)

        
        detected_face = img[int(y):int(y+h), int(x):int(x+w)]
        detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)
        detected_face = cv2.resize(detected_face, (48, 48))
        
        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)

        img_pixels /= 255 

        predictions = model.predict(img_pixels) 
        max_index = np.argmax(predictions[0])
        emotion = emotions[max_index]
        cv2.putText(img, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        
    cv2.imshow('Facial Emotion Detection',img)

    if cv2.waitKey(1) & 0xFF == ord('q'): #q to quit
        break
 
#kill open cv 
cap.release()
cv2.destroyAllWindows()